In [ ]:
import pandas as pd
from os import walk
from pathlib import Path

import pydicom

import nrrd
import numpy as np
from tqdm import tqdm

## Loading translation and encoding database.

The files of the experiment are anonymized, thus requiring a file to re-translate sed experiments.

In [ ]:
translation = pd.read_excel("../databases/WI-EXP_new.xlsx")
translation.head()

In [ ]:
translation.dtypes

## encoding and formatting nrrds for T1, T2 and T2star

In [ ]:
experiments = ["T1/Exvivo", "T1/Invivo", "T2/Exvivo", "T2/Invivo", "T2star/Exvivo", "T2star/Invivo"]
out_dir = Path("../data/processed/nrrds/")

for experiment in experiments:
    for (root, directories, file) in tqdm(walk(f"../MRI_data/{experiment}")):
        if not root == "../MRI_data/" + experiment\
              and "Research Groningen" in root\
                and not root.endswith("Groningen"):
            sequence = Path(root)
            parts = sequence.parts
            exp = parts[4]
            time = parts[-1]
            folder_name = "_".join([exp, time])
            description = translation[translation["FolderName"] == folder_name].SeriesDescription.to_string().split("#")[-1]
            ischemia = translation[translation["FolderName"] == folder_name].WI.to_string().split()[-1]
            side = translation[translation["FolderName"] == folder_name].Side.to_string().split()[-1]
            kiney_number = translation[translation["FolderName"] == folder_name].kidney_number.to_string().split()[-1]
            experiment_n = translation[translation["FolderName"] == folder_name].Exp_match.to_string().split()[-1]
            name = '_'.join([description, ischemia, side, kiney_number, experiment_n,]) + ".nrrd"
            volume = []
            for image in Path(root).glob("*.dcm"):
                image = pydicom.dcmread(image).pixel_array
                volume.append(image)
            try:
                volume = np.array(volume).transpose((1, 2, 0))
                f_out = out_dir/Path(name)
                nrrd.write(str(f_out), volume)
            except ValueError:
                print("Empty volume encountered... Will be skipping fr fr")


## Formatting T1, T2, T2star

In [ ]:
out_dir = Path("../data/processed/nrrds/")
place = {"l": "left", "r": "right", "both": "both"}
for (root, dir, files) in walk(out_dir):
    for file in files:
        if not "DWI" in file and not "ASL" in file and not "Store" in file:
            file = Path(file)
            rename_file = out_dir/file
            stem = file.stem
            parts = stem.split("_")[1:]
            time = parts[2].strip("min")

            if not parts[3] == "Invivo":
                vivo = "exvivo"
            else:
                vivo = "invivo"
                time = "-10"
            placement = place[parts[1]]    
            experiment = parts[-1]
            kidney = parts[-2]
            ischemia = parts[3].lower()
            if ischemia == "Both":
                ischemia = "invivo"
            sequence = parts[0]
            file = '_'.join([sequence, experiment, time, vivo, placement, ischemia, kidney]) + ".nrrd"
            f_out = out_dir/Path(file)
            rename_file.rename(f_out)
            

## Adding encoding for DWI and ASL

In [ ]:
out_dir = Path("../data/processed/nrrds/")
for (root, dir, files) in walk(out_dir):
    for file in files:
        if file.startswith("DWI") or file.startswith("ASL"):
            file = Path(file)
            rename_file = out_dir/file
            stem = file.stem
            parts = stem.split("_")
            exp = int(parts[1])
            lr = parts[-1].capitalize()

            ischemia, kidney = translation[(translation.Exp_match == exp) & 
                                           (translation.Side == lr)]\
                                            [["WI", "kidney_number"]]\
                                            .drop_duplicates()\
                                                .values[0]
            f_out = out_dir/Path('_'.join(parts + [ischemia.lower(), kidney]) + ".nrrd")
            rename_file.rename(f_out)